## Callbacks: connect dash core and HTML components
### Create a function that will create output in dashboard
- Decorate function with @app.callback decorator
- Set an input and output to compnent id 

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [10]:
app = dash.Dash()

app.layout = html.Div([
    dcc.Input(id = "my-id",value = "Initial Text",type = "text"), #core component, allows us to input text
    html.Div(id = "my-div") #dcc.input is different from dash dependencies input
])

#we want to connect the dcc.Input to the output in the html.Div proceeding it
#to connect text box input in core component inside of the div 
@app.callback(Output(component_id = "my-div", component_property = "children"),
             [Input(component_id="my-id",component_property="value")]) #input is a list
#we will have an output, and property we want to affect is children property

def update_output_div(input_value):
    return "You entered: {}".format(input_value) #takes input value and returns what you entered

if __name__ == "__main__":
    app.run_server()

 * Serving Flask app "dash" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Aug/2019 12:50:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 12:50:07] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 12:50:07] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 12:50:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 12:50:07] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 12:50:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 12:50:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 12:50:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 12:50:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 12:50:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 12:50:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 12:50:12] "POST /_dash-update-component HTTP/1.1" 2

## Callbacks for Graphs

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd

df = pd.read_csv('gapminderDataFiveYear.csv')
df.head()

,country,year,pop,continent,lifeExp,gdpPercap
0,Afghanistan,1952,8425333.0,Asia,28.801,779.445314
1,Afghanistan,1957,9240934.0,Asia,30.332,820.853030
2,Afghanistan,1962,10267083.0,Asia,31.997,853.100710
3,Afghanistan,1967,11537966.0,Asia,34.020,836.197138
4,Afghanistan,1972,13079460.0,Asia,36.088,739.981106


In [13]:
app = dash.Dash()

year_options = [] #collect dropdown optionsb
for year in df['year'].unique():
    year_options.append({'label':str(year),'value':year}) #for every unique year, we're grabbing year options
#append to the list a dictionary which has label and value
#so there are basically multiple lines of code in options parameter
    
app.layout = html.Div([
    dcc.Graph(id='graph'), 
    dcc.Dropdown(id='year-picker',
                 options=year_options,
                 value=df['year'].min()) #default value
])

#want to connect input from dropdown menu to output of the graph
@app.callback(Output('graph', 'figure'),
              [Input('year-picker', 'value')])

#based off input year, we will filter df for that
def update_figure(selected_year):
    filtered_df = df[df['year'] == selected_year]
    traces = []
    #filter by continent
    for continent_name in filtered_df['continent'].unique():
        df_by_continent = filtered_df[filtered_df['continent'] == continent_name]
        traces.append(go.Scatter(
            x=df_by_continent['gdpPercap'],
            y=df_by_continent['lifeExp'],
            text=df_by_continent['country'],
            mode='markers',
            opacity=0.7,
            marker={'size': 15},
            name=continent_name
        ))

    return {
        'data': traces,
        'layout': go.Layout(
            xaxis={'type': 'log', 'title': 'GDP Per Capita'},
            yaxis={'title': 'Life Expectancy'},
            hovermode='closest'
        )
    }

if __name__ == '__main__':
    app.run_server()
    
#read data
#create app
#create year options
#app layout is a graph and a dropdown

 * Serving Flask app "dash" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Aug/2019 13:02:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 13:02:17] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 13:02:17] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 13:02:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 13:02:18] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 13:02:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 13:02:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 13:02:28] "POST /_dash-update-component HTTP/1.1" 200 -


## Multiple Inputs

In [ ]:
df = pd.read_csv("mpg.csv")

app=dash.Dash()
features = df.columns #list of actual columns

#put each dropdown into own separate div so can adjust its style easily
app.layout = html.Div([
                    html.Div([           #first dropdown
                        dcc.Dropdown(id="xaxis",
                                    options=[{"label":i,"value":i} for i in features],
                                    value="displacement")
                    ],style = {"width":"48%","display":"inline-block"}), #width of div and inline block allows the two dropdowns to be next to each other   
                    html.Div([           #second dropdown
                        dcc.Dropdown(id="yaxis",
                                    options=[{"label":i,"value":i} for i in features],
                                    value="mpg")
                    ], style = {"width":"48%","display":"inline-block"}),
    dcc.Graph(id="feature-graphic") #our actual graph
],style={"padding":10})

@app.callback(
    Output('feature-graphic', 'figure'),
    [Input('xaxis', 'value'),
     Input('yaxis', 'value')])

def update_graph(xaxis_name, yaxis_name):
    return {
        'data': [go.Scatter(
            x=df[xaxis_name],
            y=df[yaxis_name],
            text=df['name'], #if you over something, you will see the name of the particular car model
            mode='markers',
            marker={
                'size': 15,
                
                'line': {'width': 0.5, 'color': 'white'}
            }
        )],
        'layout': go.Layout(title="My Dashboard for MPG",
            xaxis={'title': xaxis_name.title()},
            yaxis={'title': yaxis_name.title()},
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )
    }

if __name__ == '__main__':
    app.run_server()
    
#do imports, create app, add styling
#the function passes what the data we are referencing


 * Serving Flask app "dash" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Aug/2019 14:06:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 14:06:28] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 14:06:28] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 14:06:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 14:06:28] "GET /favicon.ico HTTP/1.1" 200 -


## Multiple Outputs with Multiple Inputs

In [3]:
filtered_df = df[df['year'] ==1957]

traces = []
    #filter by continent
for continent_name in filtered_df['continent'].unique():
    df_by_continent = filtered_df[filtered_df['continent'] == continent_name]
    traces.append(go.Scatter(
            x=df_by_continent['gdpPercap'],
            y=df_by_continent['lifeExp'],
            text=df_by_continent['country'],
            mode='markers',
            opacity=0.7,
            marker={'size': 15},
            name=continent_name
        ))
    
print(traces)

[Scatter({
    'marker': {'size': 15},
    'mode': 'markers',
    'name': 'Asia',
    'opacity': 0.7,
    'text': array(['Afghanistan', 'Bahrain', 'Bangladesh', 'Cambodia', 'China',
                   'Hong Kong, China', 'India', 'Indonesia', 'Iran', 'Iraq', 'Israel',
                   'Japan', 'Jordan', 'Korea, Dem. Rep.', 'Korea, Rep.', 'Kuwait',
                   'Lebanon', 'Malaysia', 'Mongolia', 'Myanmar', 'Nepal', 'Oman',
                   'Pakistan', 'Philippines', 'Saudi Arabia', 'Singapore', 'Sri Lanka',
                   'Syria', 'Taiwan', 'Thailand', 'Vietnam', 'West Bank and Gaza',
                   'Yemen, Rep.'], dtype=object),
    'x': array([   820.8530296,  11635.79945  ,    661.6374577,    434.0383364,
                   575.9870009,   3629.076457 ,    590.061996 ,    858.9002707,
                  3290.257643 ,   6229.333562 ,   5385.278451 ,   4317.694365 ,
                  1886.080591 ,   1571.134655 ,   1487.593537 , 113523.1329   ,
                  6089.78